In [1]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="config/settings.env")



True

## TEST BLOB UPLOAD

In [2]:
from ingestion.blob_upload import upload_to_blob

In [3]:
blob_url=upload_to_blob("data/Job Offer-Gen AI Developer Specialist (1).pdf","docs","offer.pdf")

[2025-07-09 00:17:17,558] INFO - Inputs: file_path=data/Job Offer-Gen AI Developer Specialist (1).pdf, container_name=docs, blob_name=offer.pdf
[2025-07-09 00:17:17,575] INFO - Connection string: Set
[2025-07-09 00:17:17,580] INFO - Blob service account name: myblob111
[2025-07-09 00:17:17,714] INFO - Blob client URL: https://myblob111.blob.core.windows.net/docs/offer.pdf
[2025-07-09 00:17:21,432] INFO - Uploaded offer.pdf and generated SAS URL: https://myblob111.blob.core.windows.net/docs/offer.pdf?se=2025-07-08T19%3A17%3A21Z&sp=r&sv=2025-05-05&sr=b&sig=hB/RydZMffTsoLAj3E9lm%2BiVyuuTg7920PR0icKgYOY%3D


In [4]:
blob_url

'https://myblob111.blob.core.windows.net/docs/offer.pdf?se=2025-07-08T19%3A17%3A21Z&sp=r&sv=2025-05-05&sr=b&sig=hB/RydZMffTsoLAj3E9lm%2BiVyuuTg7920PR0icKgYOY%3D'

## TEST DOCUMENT INTEL

In [16]:
from ingestion.parse import parse_document

In [17]:
text=parse_document(blob_url)

[2025-07-08 23:33:13,543] INFO - Text extraction complete


In [18]:
text

'HEXAWARE\n26-Apr-2025\nDebdoot Sen\nKolkata\nDear Debdoot,\nThis is with reference to the discussions you had with us in the last few days based on\nwhich we are now pleased to offer you a position with us as an employee at our Pune office.\nYou will be designated as Sr. Software Engineer at G4.\nOn joining, you will be called upon to sign an Appointment Letter specifying the terms\nand conditions of employment as applicable from time to time. The Appointment Letter will\ncome into effect not later than 26-Apr-2025. The compensation would be as per the Annexure\ngiven along with this Letter of Intent. Your compensation shall be reviewed annually.\nYour appointment is transferable and the Company at its discretion may transfer you to\nany other department or to any place in India or outside India and as such you may, at any time,\nbe transferred to any of the offices of the Company, its associates, organizations with whom the\ncompany has transactions, whether the office, subsidiary, a

## TEST CHUNKING

In [19]:
from ingestion.chunk import chunk_text

In [20]:
chunks=chunk_text(text)

[2025-07-08 23:33:13,728] INFO - Text chunking complete. 24 chunks created.


In [21]:
chunks

['HEXAWARE\n26-Apr-2025\nDebdoot Sen\nKolkata\nDear Debdoot,\nThis is with reference to the discussions you had with us in the last few days based on\nwhich we are now pleased to offer you a position with us as an employee at our Pune office.\nYou will be designated as Sr. Software Engineer at G4.\nOn joining, you will be called upon to sign an Appointment Letter specifying the terms\nand conditions of employment as applicable from time to time. The Appointment Letter will',
 'come into effect not later than 26-Apr-2025. The compensation would be as per the Annexure\ngiven along with this Letter of Intent. Your compensation shall be reviewed annually.\nYour appointment is transferable and the Company at its discretion may transfer you to\nany other department or to any place in India or outside India and as such you may, at any time,\nbe transferred to any of the offices of the Company, its associates, organizations with whom the',
 'company has transactions, whether the office, subsid

## TEST Embedding 

In [ ]:
from backend.embed import generate_chunk_embeddings

In [ ]:
from backend.embed_model import AzureEmbedder
from openai import AzureOpenAI

In [ ]:
em=AzureEmbedder()
em.embed_query("How are you")

In [ ]:
from openai import AzureOpenAI
import os

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=os.getenv("AZURE_OPENAI_EMBEDDING_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.embeddings.create(
    input="How are you",
    model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
)

print(response.data[0].embedding)


In [ ]:
from backend.embed import create_embeddings_index

## TESTING EMBEDDING + DATA UPSERT TO AI-SEARCH

In [22]:
from backend.embed import create_embeddings_index



create_embeddings_index(chunks)


[2025-07-08 23:33:23,534] INFO - Starting embedding + indexing pipeline...
[2025-07-08 23:33:24,654] INFO - Generating embeddings for all text chunks...
[2025-07-08 23:33:32,106] INFO - Generated embeddings for 24 chunks.
[2025-07-08 23:33:32,106] INFO - Uploading 24 documents to Azure AI Search index 'multimodal-index'...
[2025-07-08 23:33:34,424] INFO - Documents uploaded successfully.
[2025-07-08 23:33:34,431] INFO - Embedding pipeline completed successfully.


## FINAL RETRIEVAL USING LLM RESPONSE

In [25]:
from backend.search import search_query
search_query("What is the employee name and date of joining?")

'Employee Name: Debdoot Sen  \nDate of Joining: 26-Apr-2025  '